In [2]:
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt
import numpy as np

In [5]:
Batch_Size=31
Image_Size=256
Channels=3
Epochs=30

In [6]:
dataset= tf.keras.preprocessing.image_dataset_from_directory(
    "D:/VSCode/AI & ML Project/Dataset/Apple",
    shuffle=True,
    image_size=(Image_Size,Image_Size),
    batch_size= Batch_Size
)

Found 3171 files belonging to 4 classes.


In [43]:
class_names= dataset.class_names
class_names

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy']

In [7]:
def get_dataset_split(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    assert(train_split+val_split+test_split)==1
    ds_size= len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=10)
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    
    return train_ds,test_ds,val_ds


In [8]:
train_ds,test_ds,val_ds = get_dataset_split(dataset)

In [46]:
print(len(train_ds),len(test_ds),len(val_ds))

82 11 10


In [47]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)#loading to cache memory to reduce training time
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [48]:
resize_and_rescale =tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(Image_Size,Image_Size),#resizing
    layers.experimental.preprocessing.Rescaling(1.0/255)#normalizing
])

In [49]:
data_augmentation= tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomContrast(0.2),
    layers.experimental.preprocessing.RandomZoom(0.3)
])

In [50]:
input_shape=(Batch_Size,Image_Size,Image_Size,Channels)
n_classes=4
model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape),#relu makes model non linear by converting negative values in feature map to 0
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax'),
])
model.build(input_shape=input_shape)

In [51]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [52]:
data=model.fit(
    train_ds,
    batch_size=Batch_Size,
    validation_data=val_ds,
    verbose=1,
    epochs=Epochs
)

Epoch 1/30


82/82 [==============================] - 107s 1s/step - loss: 1.1002 - accuracy: 0.5337 - val_loss: 0.9654 - val_accuracy: 0.5774
Epoch 2/30
82/82 [==============================] - 115s 1s/step - loss: 0.8512 - accuracy: 0.6317 - val_loss: 0.9791 - val_accuracy: 0.6935
Epoch 3/30
82/82 [==============================] - 96s 1s/step - loss: 0.8308 - accuracy: 0.6381 - val_loss: 0.9299 - val_accuracy: 0.6419
Epoch 4/30
82/82 [==============================] - 96s 1s/step - loss: 0.6696 - accuracy: 0.7246 - val_loss: 0.6666 - val_accuracy: 0.7387
Epoch 5/30
82/82 [==============================] - 94s 1s/step - loss: 0.5978 - accuracy: 0.7603 - val_loss: 0.9870 - val_accuracy: 0.7097
Epoch 6/30
82/82 [==============================] - 94s 1s/step - loss: 0.5331 - accuracy: 0.7857 - val_loss: 0.7154 - val_accuracy: 0.7742
Epoch 7/30
82/82 [==============================] - 95s 1s/step - loss: 0.5749 - accuracy: 0.7694 - val_loss: 0.9874 - val_accuracy: 0.6903
Epoch 8/30
82/82 [===========

In [57]:
model.save("D:/VSCode/AI & ML Project/models/apple")

INFO:tensorflow:Assets written to: D:/VSCode/AI & ML Project/models/apple\assets


INFO:tensorflow:Assets written to: D:/VSCode/AI & ML Project/models/apple\assets


In [23]:
scores=model.evaluate(test_ds)

11/11 [==============================] - 6s 417ms/step - loss: 0.2076 - accuracy: 0.9355


In [18]:
model=tf.keras.models.load_model("D:/VSCode/AI & ML Project/models/apple")

In [22]:
scores=model.evaluate(test_ds)

11/11 [==============================] - 6s 410ms/step - loss: 0.1958 - accuracy: 0.9443


In [61]:
filename = "D:/VSCode/AI & ML Project/Dataset/Apple/Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417.JPG"
# AI & ML Project/Dataset/Tomato/Tomato___Bacterial_spot/00a7c269-3476-4d25-b744-44d6353cd921___GCREC_Bact.Sp 5807.JPG
# AI & ML Project/Dataset/Apple/Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417.JPG
def process_image(image_path):
    # Load and preprocess the image for your model
    img = Image.open(image_path)
    img = img.resize((256, 256))  # adjust the size according to your model's input
    img_array = np.array(img) / 255.0  # normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
    return img_array

img_array = process_image(filename)
batch_prediction= model.predict(img_array)
print("Predicted Disease",class_names[np.argmax(batch_prediction[0])])

1/1 [==============================] - 0s 26ms/step
Predicted Disease Apple___healthy
